# 1. Write a Function for calculating the Parametric VaR of DLF (40%), NTPC (20%) And HDFC Bank (40%). Take historical data for last 500 days before 1st Oct, 2019 and a 99% VaR confidence level. 

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.stats as stats
import datetime as dt

C:\Users\koushik\AppData\Roaming\Python\Python311\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


## Function to get stock data

In [2]:
def get_stock_data(stockList, startDate, endDate):
    stockData = yf.download(stockList, start=startDate, end=endDate)
    stockData = stockData['Adj Close']
    return stockData

In [3]:
stock_list = ['DLF.NS', 'NTPC.NS', 'HDFCBANK.NS']
wtgs = [0.4, 0.2, 0.4]
df = get_stock_data(stockList=stock_list, startDate='2017-09-15', endDate='2019-10-1')
data = df.copy(deep=True)

[*********************100%%**********************]  3 of 3 completed


## Function to get Log Returns, Mean and Standard Deviation of the data

In [4]:
def portfMetrics(weights, data):
    # Caculating the Log Returns of the Portfolio
    returns = np.log(data).diff() 
    returns = returns[stock_list]
    returns.dropna(inplace=True)
    
    portReturns = np.sum(weights * returns, axis=1)
    # Calulating mean and standard deviation for returns
    meanReturns = portReturns.mean()
    wgts = np.array(weights).reshape(-1, 1)
    cov_var = returns.cov()
    stdDevReturns = np.sqrt(wgts.T.dot(cov_var).dot(wgts))    

    return portReturns, meanReturns, stdDevReturns

In [5]:
PortfolioReturns, Mean_Returns, StdDeviation_Returns = portfMetrics(wtgs, data)
Mean_Returns


5.989567248881189e-05

## Function to calculate the Parametric VaR

In [6]:
def parametric_VaR1(meanReturns, stdDevReturns, conf_Interval):

    parametricVaR = meanReturns - stats.norm.ppf(conf_Interval/100) *stdDevReturns 

    return parametricVaR 

In [8]:
PortfolioReturns = pd.DataFrame({'returns' : PortfolioReturns * 1e6})
PortfolioReturns

,returns
Date,
2017-09-18,8029.811099
2017-09-19,-3536.004186
2017-09-20,-3606.647558
2017-09-21,-11671.258134
2017-09-22,-31688.423460
...,...
2019-09-24,-3730.958569
2019-09-25,-30002.188112
2019-09-26,15422.702182


In [7]:
conf_Interval = 99
print("Parametric VaR of " + str(conf_Interval) + "% VaR is : " + str(parametric_VaR1(Mean_Returns, StdDeviation_Returns, conf_Interval) * 1e6))

Parametric VaR of 99% VaR is : [[-32188.90765552]]


## Parametric VaR of 99% VaR is : 32188.90765552